|data|lines|
|:---|---:|
|train|184,903,890|
|test |1,879,069|

train が test の 100 倍ある。メモリゲーっぽい。

Bosch コンペの時に、明らかにクソなデータを間引くことで学習を高速化するテクがあったのを思い出した。

In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns

train_columns = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_columns  = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

/home/kenkou_n/.pyenv/versions/3.6.4/envs/talking-data/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ModuleNotFoundError: No module named 'seaborn'

In [5]:
%%time
train=pd.read_csv("../data/train.csv", usecols=train_columns, dtype=dtypes)

CPU times: user 1min 35s, sys: 3.1 s, total: 1min 38s
Wall time: 1min 38s


In [14]:
%%time
test=pd.read_csv("../data/test.csv", usecols=test_columns, dtype=dtypes)

CPU times: user 11.2 s, sys: 652 ms, total: 11.9 s
Wall time: 16.6 s


In [6]:
train.to_hdf("../data/train.hdf", "train")

In [15]:
test.to_hdf("../data/test.hdf", "test")

In [4]:
%%time
train=pd.read_hdf("../data/train.hdf", "train")

CPU times: user 4.61 s, sys: 3.08 s, total: 7.69 s
Wall time: 7.7 s


In [16]:
%%time
test=pd.read_hdf("../data/test.hdf", "test")

CPU times: user 520 ms, sys: 380 ms, total: 900 ms
Wall time: 897 ms


In [5]:
train

,ip,app,device,os,channel,click_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,0
1,17357,3,1,19,379,2017-11-06 14:33:34,0
2,35810,3,1,13,379,2017-11-06 14:34:12,0
3,45745,14,1,13,478,2017-11-06 14:34:52,0
4,161007,3,1,13,379,2017-11-06 14:35:08,0
5,18787,3,1,16,379,2017-11-06 14:36:26,0
6,103022,3,1,23,379,2017-11-06 14:37:44,0
7,114221,3,1,19,379,2017-11-06 14:37:59,0
8,165970,3,1,13,379,2017-11-06 14:38:10,0
9,74544,64,1,22,459,2017-11-06 14:38:23,0


## 思考停止 count
正例は全体の 0.2% しかない。

In [8]:
train["is_attributed"].value_counts()

0    184447044
1       456846
Name: is_attributed, dtype: int64

In [10]:
train["ip"].value_counts()

5348      1238734
5314      1171448
73516      770451
73487      763854
53454      498186
114276     427453
26995      401495
95766      378693
17149      310996
105475     302192
100275     276799
43793      261970
105560     260049
86767      257649
111025     247187
137052     217614
201182     212448
5178       211556
49602      200053
5147       197994
48282      188092
48212      185885
48240      182754
48170      180609
123994     176285
209663     173423
93587      171417
84896      170166
45745      169149
119369     159276
           ...   
287517          1
322237          1
191100          1
322238          1
275703          1
358930          1
275702          1
252062          1
144444          1
219639          1
322200          1
219638          1
144579          1
219598          1
191066          1
275458          1
358932          1
228052          1
239835          1
219593          1
25479           1
219594          1
322193          1
252079          1
174284    

In [11]:
train["channel"].value_counts()

280    15065927
245     8873025
107     8457471
477     7188340
134     5924250
259     5787004
153     5537977
265     5446388
178     5271408
121     4692438
205     4359180
145     3583945
442     3574611
459     3477150
379     3402636
439     2901762
489     2711240
128     2703163
135     2663913
466     2655396
237     2632794
480     2618441
469     2588380
140     2562964
122     2526161
219     2349454
137     2302316
232     2275338
435     2192870
101     2147174
         ...   
223         118
225          96
233          61
14           22
471          20
458          16
251          14
248          14
256          13
169          13
162          12
238           9
29            6
473           5
475           3
142           2
476           2
165           2
149           2
470           1
146           1
434           1
422           1
394           1
354           1
227           1
221           1
217           1
172           1
500           1
Name: channel, Length: 2

In [12]:
train["app"].value_counts()

3      33911780
12     24179003
2      21642136
9      16458268
15     15958970
18     15756587
14     10027169
1       5796274
13      4329409
8       3731948
21      3616407
11      3466971
26      3126136
23      2675259
6       2464136
64      1893969
7       1764954
20      1758934
25      1467907
28      1311496
27      1296189
24      1259100
19       922902
17       797335
22       684604
10       684043
29       652090
32       485426
5        375533
151      188490
         ...   
678           1
679           1
558           1
681           1
404           1
653           1
684           1
410           1
687           1
608           1
689           1
691           1
673           1
671           1
571           1
669           1
668           1
667           1
572           1
665           1
664           1
578           1
661           1
580           1
659           1
582           1
657           1
656           1
655           1
626           1
Name: app, Length: 706, 